In [5]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/latest/"
train = pd.read_csv(data+'train_0.csv')
#train['item_category'] = train.item_category.astype('category')
test = pd.read_csv(data+'test_0.csv')
#test['item_category'] = test.item_category.astype('category')

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 2,
    'min_data_in_leaf': 400,
    'feature_fraction': 1,
    'bagging_fraction': 0.91,
    'bagging_freq': 1,
    'verbose': 0
}

In [3]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 40,
    'min_data_in_leaf': 90,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.96,
    'bagging_freq': 1,
    'lambda_l1': 0,
    'verbose': 0
}

In [2]:
# wish
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 8,
    'min_data_in_leaf': 3300,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.97,
    'bagging_freq': 1,
    'lambda_l1': 0,
    'verbose': 0
}

In [3]:
X = lgb.Dataset(train.drop(['user_id', 'work_id', 'rating'], axis=1), train['rating'])
gbdt = lgb.train(params, X, num_boost_round=98, valid_sets=[X], verbose_eval=False)
Y = gbdt.predict(test.drop(['user_id', 'work_id'], axis=1))

In [2]:
# fm training
from scipy.sparse import hstack
from fastFM import sgd
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_svmlight_file

path = "/mnt/d/Data/mangaki-data-challenge/libfm/"

params = {
    'n_iter':340000, 
    'init_stdev':0.01,
    'l2_reg_w':0.00025,
    'l2_reg_V':0.5,
    'rank':1,
    'step_size':0.02
}

train, ty = load_svmlight_file(path+"train_0.csv", zero_based=True)
test, _ = load_svmlight_file(path+"test_0.csv")
fm = sgd.FMClassification(**params)
fm.fit(train, np.require(ty*2-1, dtype=np.int))
Y = fm.predict_proba(test)

In [6]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=0.5, penalty='l1')

X = train.ix[:, 3:].values
Y = train.ix[:, 2].values
lr.fit(X, Y)
Xt = test.ix[:, 2:].values
Yt = lr.predict_proba(Xt)

In [8]:
test = pd.read_csv('/mnt/d/Data/mangaki-data-challenge/test.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16
})
#test['prob_willsee'] = (pgbdt*0.804944+prf*0.802744)/(0.802744+0.804944)
test['prob_willsee'] = Yt[:, 1]

In [6]:
pgbdt=Y

In [3]:
prf=Y

In [9]:
test.to_csv("/mnt/d/Data/mangaki-data-challenge/result_10_01_01.csv", index=False)

In [7]:
Yt

array([[ 0.55713671,  0.44286329],
       [ 0.27377348,  0.72622652],
       [ 0.79345197,  0.20654803],
       ..., 
       [ 0.24459636,  0.75540364],
       [ 0.12291004,  0.87708996],
       [ 0.74888598,  0.25111402]])

In [9]:
pt = pd.read_csv(data+"../libfm/test_result.csv", header=None, names=['prob_willsee'])

In [11]:
test['prob_willsee']=pt

In [16]:
(fm.predict_proba(test)>0.99).sum()

92698

In [20]:
_

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [30]:
train

<11112x11891 sparse matrix of type '<class 'numpy.float64'>'
	with 146128 stored elements in Compressed Sparse Row format>

In [31]:
test

<100015x11892 sparse matrix of type '<class 'numpy.float64'>'
	with 1315262 stored elements in Compressed Sparse Row format>

In [23]:
test[0,:].data

array([ 1.        ,  1.        ,  0.72826087,  0.15217391,  2.60869565,
        0.17366651,  0.27941176,  0.2254902 ,  0.00980392,  0.48529412,
        1.96568627,  0.18466436])

In [28]:
test[0,:].indptr

array([ 0, 12], dtype=int32)

In [25]:
train[0, :].data

array([ 1.        ,  1.        ,  0.39189189,  0.39189189,  0.04054054,
        0.17567568,  0.52027027,  0.97582781,  0.07478632,  0.70299145,
        0.04273504,  0.17948718,  0.6784188 ,  0.66357103])